# Loading and visualizin Smart Scan Data
In this notebook we will load and visualize the data recorded using the gaussian process driven data aquisition system "Smart Scan".

To run this notebook, please install the smartscan library, as described in the readme.

The data can be obtained from the following link: [Smart Scan Data](https://drive.google.com/drive/folders/1Q6Q6Q6Z

In [ ]:
from itertools import product

import matplotlib.pyplot as plt
import smartscan as ss
import xarray as xr
from tqdm.auto import tqdm


# Loading smart scan 
The loader provided in the smartscan library loads the data as an `xarray.Dataset` object, containing the recoded spectra and related coordinates, as well as the reduced *tasks* as used to drive the Gaussian Process Regression.

In [ ]:
s2D = ss.load_smartscan("smartscan2D.h5")
# shift coordinate system
s2D.coords["x"] = -(s2D.x + 3300) / 1000
s2D.coords["y"] = -(s2D.y + 3300) / 1000
s2D.x.attrs["units"] = "mm"
s2D.y.attrs["units"] = "mm"
s2D

it is often useful to handle this data as a pandas DataFrame. This can be done by calling the to_pandas method on the smartscan object, after isolating the higher dimensional data and relative dimensions (`energy`, `phi` and the `spectra`)

In [ ]:
df2D = s2D.drop_vars(["spectra", "energy", "phi"]).to_dataframe()
df2D


## visualize the acquired map

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ss.vornoi_plot(
    points=df2D[["y", "x"]].values,
    values=df2D["curvature"],
    ax=ax,
)
ax.set_xlabel("x [mm]")
ax.set_ylabel("y [mm]")


# load raster maps

In [ ]:
raster2D = ss.io.load_raster_scan("raster2D_theta_2.h5")
raster2D.coords["x"] = -(raster2D.x + 3300) / 1000
raster2D.coords["y"] = -(raster2D.y + 3300) / 1000
raster2D = raster2D.sortby(["y", "x"])
raster2D.x.attrs["units"] = "mm"
raster2D.y.attrs["units"] = "mm"
raster2D


# compute curvature
we now compute the curvature task as used in the smartscan2D file. We pick the ROI usef from the metadata of the smartscan2D.

In [ ]:
prod = list(
    product(
        range(len(raster2D.x.values)),
        range(len(raster2D.y.values)),
    )
)
raster2d_curvature = xr.zeros_like(raster2D.isel({"energy": 0, "phi": 0})).drop_vars(
    ["energy", "phi"]).assign_coords({'theta': 2})
raster2d_curvature.name = 'curvature'
for x, y in tqdm(prod):
    raster2d_curvature[x, y] = ss.tasks.curvature(
        raster2D.isel({"x": x, "y": y}).isel(s2D.attrs["roi_dict"])
    )
raster2d_curvature

# overlay the smart scan points

In [ ]:
fig, ax = plt.subplots(1, 1)
# normalize curvature
raster2d_curvature = raster2d_curvature / raster2d_curvature.max()
raster2d_curvature.plot(
    ax=ax,
    alpha=0.7,
)
ax.set_aspect("equal")
plt.scatter(df2D["y"], df2D["x"], c=df2D["curvature"], marker=".", cmap="viridis")
